In [7]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd

# Load the built-in MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k')

# Split into training and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build and train the model (SVD = Matrix Factorization)
model = SVD()
model.fit(trainset)

# Predict ratings for the test set
predictions = model.test(testset)

# Evaluate the model
print("\n✅ Model Evaluation:")
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Show sample predictions
print("\n🔍 Sample Predictions:")
for pred in predictions[:5]:
    print(f"User {pred.uid} rated Item {pred.iid} as {pred.r_ui} → Predicted: {round(pred.est, 2)}")

# Cross-validate for more robust metrics
print("\n📊 Cross-Validation Results:")
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


# 💡 Generate Top-5 Recommendations for a Specific User
from surprise.model_selection import train_test_split
from collections import defaultdict

uid = str(196)  # Example user ID (must be a string)
all_items = trainset.all_items()
all_item_inner_ids = [iid for iid in trainset.all_items()]
all_item_raw_ids = [trainset.to_raw_iid(iid) for iid in all_item_inner_ids]

# Get items the user has already rated
user_rated = set([j for (j, _) in trainset.ur[trainset.to_inner_uid(uid)]])

# Predict for all unrated items
unrated_items = [iid for iid in all_item_inner_ids if iid not in user_rated]
predictions = [model.predict(uid, trainset.to_raw_iid(iid)) for iid in unrated_items]

# Sort by estimated rating
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

# Print recommendations
print(f"\n🎯 Top 5 Movie Recommendations for User {uid}:")
for pred in recommendations:
    print(f"Movie ID: {pred.iid} → Predicted Rating: {round(pred.est, 2)}")



✅ Model Evaluation:
RMSE: 0.9362
MAE:  0.7375

🔍 Sample Predictions:
User 907 rated Item 143 as 5.0 → Predicted: 5
User 371 rated Item 210 as 4.0 → Predicted: 4.25
User 218 rated Item 42 as 4.0 → Predicted: 3.7
User 829 rated Item 170 as 4.0 → Predicted: 3.99
User 733 rated Item 277 as 1.0 → Predicted: 2.74

📊 Cross-Validation Results:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9460  0.9382  0.9370  0.9265  0.9343  0.9364  0.0063  
MAE (testset)     0.7443  0.7397  0.7383  0.7302  0.7375  0.7380  0.0046  
Fit time          1.66    1.82    1.81    1.81    1.82    1.79    0.06    
Test time         0.49    0.32    0.32    0.45    0.33    0.38    0.07    

🎯 Top 5 Movie Recommendations for User 196:
Movie ID: 318 → Predicted Rating: 4.51
Movie ID: 357 → Predicted Rating: 4.49
Movie ID: 191 → Predicted Rating: 4.46
Movie ID: 190 → Predicted Rating: 4.45
Movie ID: 480 → Predicted Ratin